In [ ]:
from geopy.geocoders import Nominatim
import geopy
import pandas as pd
import geopandas as gpd
from geopy.extra.rate_limiter import RateLimiter
import time

In [ ]:
df = pd.read_csv("data/processed/processed_municip_kWh.csv", encoding="utf-8")
# df.head() 

In [ ]:
df_annual = df[df['Month'].str.contains("Annual", na=False)]
df_annual['address'] = df_annual['Municipality'] + ', ' + df_annual['Province']
# df_annual.head() 

In [ ]:
address = df_annual.iloc[0]["address"]
address

In [ ]:
# try running this block first to see if can create latitude and longtitude of one single location 
loc = Nominatim(user_agent="Geopy Library")
getLoc = loc.geocode(address)

print(getLoc.address)
print("Latitude = ", getLoc.latitude, "\n")
print("Longitude = ", getLoc.longitude)

In [ ]:
# then run this 

if 'latitude' not in df_annual.columns:
    df_annual['latitude'] = pd.NA
if 'longitude' not in df_annual.columns:
    df_annual['longitude'] = pd.NA

# Set up the geolocator with rate limiting
geolocator = geopy.Nominatim(user_agent="myGeocoder")
rate_limiter = RateLimiter(geolocator.geocode, min_delay_seconds=1)  # adjust min_delay_seconds as needed

for index, row in df_annual.iterrows():
    if pd.isna(row['latitude']) or pd.isna(row['longitude']):  # Check if geocoding is needed
        try:
            location = rate_limiter(row['address'])
            if location:
                df_annual.at[index, 'latitude'] = location.latitude
                df_annual.at[index, 'longitude'] = location.longitude
        except geopy.exc.GeocoderServiceError as e:
            print(f"Error geocoding {row['address']}: {e}")

print(df_annual.head())  # Show the first few rows to check